# Top TRB Holders

## Imports

In [2]:
from decouple import config
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import web3
from web3 import Web3

## Global variables

In [4]:
mainnet_url = config('MAINNET_ACCESS_URL')
w3 = Web3(Web3.HTTPProvider(mainnet_url, request_kwargs={'timeout':60}))
print(w3.isConnected())

True


In [52]:
last_block = w3.eth.blockNumber
start_block = last_block - 2000

In [53]:
path = os.path.join('..', 'abi', 'abi.json')
with open(path) as f:
    abi = f.read()
address = Web3.toChecksumAddress("0x0ba45a8b5d5575935b8158a88c631e9f9c95a2e5")
tellor_contract = w3.eth.contract(address=address, abi=abi)

## Filtering

In [54]:
fil = w3.eth.filter({
    'fromBlock': start_block,
    'toBlock': last_block,
    'address': address
})

In [55]:
tx_list = fil.get_all_entries()

In [56]:
print(tx_list[0])

AttributeDict({'address': '0x0Ba45A8b5d5575935B8158a88C631E9F9C95a2e5', 'topics': [HexBytes('0xbeb3b9f46c8d7bb00c873fca10d307538df350866d25f891ffb395147ddbdc45'), HexBytes('0xa248cdebe287dbd3a5556de9e63420213b2997451246eeb2800295854f2b1fd8')], 'data': '0x0000000000000000000000000000000000000000000000000000000000000039000000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000000000000000000030000000000000000000000000000000000000000000000000000000000000032000000000000000000000000000000000000000000000000000000000000000a000000000000000000000000000000000000000000000000000000005ffedfd000000000000000000000000000000000000000000000000000000000000c706400000000000000000000000000000000000000000000000000000000000c707e00000000000000000000000000000000000000000000000000000000000c746400000000000000000000000000000000000000000000000000000000000c748800000000000000000000000000000000000000000000000000000000000c757c00000000000000000000000000000000000000000

## Retrieving Transfers and TipAdded Events

In [63]:
def update_balances(events_list, balances):
    '''Loops through DataFrame representing events
       and updates address balances'''
    for i in tqdm(events_list):
        _from = i['args']['_from']
        _to = i['args']['_to']
        _value = i['args']['_value']
        if _to not in balances:
            balances[_to] = _value
        else:
            balances[_to] += _value

        if _from not in balances:
            balances[_from] = -_value
        else:
            balances[_from] -= _value
        

In [57]:
fil = tellor_contract.events.Transfer.createFilter(fromBlock=start_block, toBlock=last_block)
transfers = fil.get_all_entries()

fil = tellor_contract.events.TipAdded.createFilter(fromBlock=start_block, toBlock=lastBlock)
tips_added = fil.get_all_entries()

In [64]:

balances = {}
update_balances(transfers, balances)
update_balances()


100%|██████████| 197/197 [00:00<00:00, 486618.31it/s]


In [59]:
balances

{'0x8EEF198A83b1b4266F5C386D8c5e400c9F7782f0': 24403333333333333330,
 '0x0Ba45A8b5d5575935B8158a88C631E9F9C95a2e5': -480094999999999999958,
 '0x0966AEb41F4a94aAB7FB595A22CAa7b64ce73aA2': -559576666666666666671,
 '0x96eB60e3AcB5D123dBdE49B51b0ddACBA7C8b64a': 23909999999999999998,
 '0x7034E17f11b382545FFd618620Ca72e271AEa5F6': 41096666666666666664,
 '0x4fB035C3AB1DDaeF58Dd3A830Ae55B3E57cce983': 43263333333333333332,
 '0x39E419bA25196794B595B2a595Ea8E527ddC9856': 43644999999999999993,
 '0xD4DA002e714a7341a7d0fB1899F8260508E42653': 0,
 '0xE1aAa4aD9657C1265f0caefe057b85d807860fc8': 0,
 '0x6efF3372fa352b239Bb24ff91b423A572347000D': 964500000000000000000,
 '0x5086980D1e667E76ec9146ad6db5A09d69074257': 54098371095493753281,
 '0x70258Aa9830C2C84d855Df1D61E12C256F6448b4': 41542743141985238509,
 '0x36fa937c8F96B10Ec48Dd97B117eaFcc934Ccd1D': 22319999999999999998,
 '0xB0f7ba3827b4a392B459355BB1dCBfcE0A4D7d96': 28246666666666666665,
 '0xF9650165aB28Abf7BB59c1DE36323F5F96504193': 34339999999999999998

In [5]:
df = pd.read_csv('../data/transfers.csv')
df.head()

,_from,_to,_value,tx_hash,address,eth_block_number,gas_price
0,0x70258Aa9830C2C84d855Df1D61E12C256F6448b4,0xac9356ea58183E32B9c8D6F8f4407843B1C8fE78,1619452823092278265,0x23d987d044cdf92f4c945a70d25d4dbfceb087eab9ef...,0x0Ba45A8b5d5575935B8158a88C631E9F9C95a2e5,11550037,125400000000
1,0xCCE8D59AFFdd93be338FC77FA0A298C2CB65Da59,0xABb63D4a0b8739Fa9B65f55A999d10c61921fAB8,59910093200000000000,0x36f250302e0fb2c5c8e636ad42f7013f4fe228d4823a...,0x0Ba45A8b5d5575935B8158a88C631E9F9C95a2e5,11550069,175000000000
2,0x70258Aa9830C2C84d855Df1D61E12C256F6448b4,0x2Cc0b05b1C4f2578550BdF5eb6eA170d1BeA2bA8,502965941025387216,0xab80c2583c9fcbee1c57630357f66963fad06a7b3679...,0x0Ba45A8b5d5575935B8158a88C631E9F9C95a2e5,11550091,131000000000
3,0x0Ba45A8b5d5575935B8158a88C631E9F9C95a2e5,0x173020C64df2A661b7C1EeAf85FB0bBB7d0D3290,7356666666666666666,0xf19813cce5f0ba090cfbc5f066b40d2e241fd3da0d73...,0x0Ba45A8b5d5575935B8158a88C631E9F9C95a2e5,11550092,78000001459
4,0x0Ba45A8b5d5575935B8158a88C631E9F9C95a2e5,0xDbC1b60fDd000F645B668d8026A28C26772A151c,7356666666666666666,0xf19813cce5f0ba090cfbc5f066b40d2e241fd3da0d73...,0x0Ba45A8b5d5575935B8158a88C631E9F9C95a2e5,11550092,78000001459
